In [1]:
#used to import data
import pandas as pd
#Evaluates the model
from sklearn.metrics import log_loss
import time #just used to see how long the model takes to run
from pytorch_tabnet.tab_model import TabNetRegressor

from sklearn.model_selection import train_test_split

In [2]:
df_11 = pd.read_csv('data/pbp/ready_to_model/10_11_pbp_cleaned.csv')
df_12 = pd.read_csv('data/pbp/ready_to_model/11_12_pbp_cleaned.csv')
df_13 = pd.read_csv('data/pbp/ready_to_model/12_13_pbp_cleaned.csv')
df_14 = pd.read_csv('data/pbp/ready_to_model/13_14_pbp_cleaned.csv')
df_15 = pd.read_csv('data/pbp/ready_to_model/14_15_pbp_cleaned.csv')
df_16 = pd.read_csv('data/pbp/ready_to_model/15_16_pbp_cleaned.csv')
df_17 = pd.read_csv('data/pbp/ready_to_model/16_17_pbp_cleaned.csv')
df_18 = pd.read_csv('data/pbp/ready_to_model/17_18_pbp_cleaned.csv')
df_19 = pd.read_csv('data/pbp/ready_to_model/18_19_pbp_cleaned.csv')
df_20 = pd.read_csv('data/pbp/ready_to_model/19_20_pbp_cleaned.csv')
df_21 = pd.read_csv('data/pbp/ready_to_model/20_21_pbp_cleaned.csv')
df = pd.concat([df_11,df_12,df_13,df_14,df_15,df_16,df_17,df_18,df_19,df_20,df_21])

In [3]:
def get_X_y(df):
    ''' This function takes in a dataframe and splits it into the X and y variables
    '''
    X = df.drop(['is_goal'], axis=1).to_numpy()
    y = df.is_goal.to_numpy()
    
    return X,y

X,y = get_X_y(df)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.50, random_state=101)

#X_train.values
y_train = y_train.reshape(-1,1)
y_val = y_val.reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [4]:
reg = TabNetRegressor(n_independent=4)
model = reg.fit(X_train, y_train, 
                eval_set = [(X_train, y_train), (X_val, y_val)],
                eval_name=['train','val'],
                eval_metric = ['logloss'],                
                max_epochs = 12)

Device used : cpu
epoch 0  | loss: 0.0645  | train_logloss: 0.2314  | val_logloss: 0.22904 |  0:01:06s
epoch 1  | loss: 0.05341 | train_logloss: 0.2091  | val_logloss: 0.20882 |  0:02:12s
epoch 2  | loss: 0.05273 | train_logloss: 0.20363 | val_logloss: 0.20282 |  0:03:17s
epoch 3  | loss: 0.05211 | train_logloss: 0.20142 | val_logloss: 0.20095 |  0:04:23s
epoch 4  | loss: 0.05185 | train_logloss: 0.2008  | val_logloss: 0.20013 |  0:05:26s
epoch 5  | loss: 0.05173 | train_logloss: 0.19984 | val_logloss: 0.19965 |  0:06:30s
epoch 6  | loss: 0.05165 | train_logloss: 0.19953 | val_logloss: 0.19959 |  0:07:34s
epoch 7  | loss: 0.05162 | train_logloss: 0.19847 | val_logloss: 0.1987  |  0:08:37s
epoch 8  | loss: 0.0515  | train_logloss: 0.19751 | val_logloss: 0.19765 |  0:09:44s
epoch 9  | loss: 0.05142 | train_logloss: 0.20016 | val_logloss: 0.19953 |  0:10:55s
epoch 10 | loss: 0.05137 | train_logloss: 0.19775 | val_logloss: 0.19758 |  0:11:58s
epoch 11 | loss: 0.05138 | train_logloss: 0.197

In [ ]:
#This is the vanilla or default version
reg = TabNetRegressor()
model = reg.fit(X_train, y_train, 
                eval_set = [(X_train, y_train), (X_val, y_val)],
                eval_name=['train','val'],
                eval_metric = ['logloss'],                
                max_epochs = 12)

In [5]:
preds = reg.predict(X_test)
print(log_loss(y_test, preds))

0.19929595650251805


In [ ]:
#after 10 epochs logloss = .197
#after 10 epochs logloss = .197
#Added a validation set, 10 epochs logloss = .198
#With the validation set and 10 epochs, logloss = .1965 <--- slight improvement
#increased the number of steps to 5, logloss = .200
#increased the number of steps to 8, logloss = .207
#increased the number of steps to 2, logloss = .1976
#using 3 as the default fir number of steps is fine 
#Setting the n_independent to 4 results in heavy overfitting